In [1]:
import os
import cv2
import torch
from torch import nn
from torchvision import datasets
from torchvision.utils import make_grid
from torchvision.io import read_image
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from torchvision.models import detection
import matplotlib.pyplot as plt
import torchvision.transforms as T
import numpy as np
import argparse
import pickle
import torch
import cv2
import torchvision
from PIL import *
import random
import skimage
from skimage.io import imread, imshow
import re
import torch
from collections import Counter

# from iou import intersection_over_union

In [2]:
folder_path = 'bike'
w = 1000
h = 200

In [3]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [6]:
model = detection.(pretrained=True)
model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [7]:
data_list = os.listdir(folder_path)
# annot_list = os.listdir(annot_path)
data =[]
pred = []
gt = []
gen_list=[]
for i in range(len(data_list)):
    img_path = os.path.join(folder_path, data_list[i])
#     anot_path = os.path.join(annot_path, annot_list[i])
    image = imread(img_path) 
    dim = (w, h)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
#     image = cv2.boxFilter(image, -1, (3, 3))
    image1 = image.transpose((2, 0, 1))    
    image1 = np.expand_dims(image1, axis=0)
    image1 = image1 / 255.0
    image1 = torch.FloatTensor(image1)

    
    predictions = model(image1)
    boxes = predictions[0]['boxes'].detach()
    boxes = boxes.cpu().numpy().astype(np.uint16).tolist()
    scores = predictions[0]['scores'].detach().cpu().numpy().tolist()
    labels = predictions[0]['labels'].detach().cpu().numpy().tolist()
    for label, box,s in zip(labels, boxes, scores):
        if s<0.4:
            continue
        pred.append([i,s,[COCO_INSTANCE_CATEGORY_NAMES[label],box]])
        cv2.rectangle(image, (box[0], box[1]), (box[2], box[3] ), color=(0,255,0), thickness=2)
        pred_class = COCO_INSTANCE_CATEGORY_NAMES[label]
        conf = "{} %".format(round(s,2))
        cv2.putText(image,pred_class, (box[0]-10, box[1]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0),thickness=1)
        cv2.putText(image,conf, (box[0]-10, box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0),thickness=1)
    cv2.imwrite('mask_bike/bike{}.png'.format(i),image)
    

In [ ]:
pred

In [ ]:
person=0
car=0
for i in range(len(pred)):
    if pred[i][2][0]=='person':
        person +=1
    if pred[i][2][0]=='car':
        car +=1
    
        

In [ ]:
person

In [ ]:
car

In [ ]:
detections = []
ground_truths = []

# Go through all predictions and targets,
# and only add the ones that belong to the
# current class c
for detection in pred:
    if detection[2][0] == 1:
        detections.append(detection)

for true_box in gt:
    for jj in range(len(true_box[1])):
        if true_box[1][jj][0] == 1:
            ground_truths.append([true_box[0],[1, true_box[1][jj][1:]]])


detections.sort(key=lambda x: x[1], reverse=True)
TP = torch.zeros((len(detections)))
FP = torch.zeros((len(detections)))


for detection_idx, detection in enumerate(detections):
    print(detection[0])
    print(detection[2][1])
    
    # Only take out the ground_truths that have the same
    # training idx as detection
    ground_truth_img = [
        bbox for bbox in ground_truths if bbox[0] == detection[0]
    ]

    num_gts = len(ground_truth_img)
    best_iou = 0

    for idx, gtr in enumerate(ground_truth_img):
        print(gtr[1][1])
#         iou = intersection_over_union(
#             torch.tensor(detection[2][1]),
#             torch.tensor(gtr[3:]),
#             box_format="midpoint",
# #         )

In [ ]:
detection[3:]

In [ ]:
for true_box in gt:
    for jj in range(len(true_box[1])):
        if true_box[1][jj][0] == c:
            ground_truths.append([true_box[0],[c, true_box[1][jj][1:]]])

In [ ]:
# list storing all AP for respective classes
average_precisions = []

# used for numerical stability later on
epsilon = 1e-6

for c in range(5):
    detections = []
    ground_truths = []

    # Go through all predictions and targets,
    # and only add the ones that belong to the
    # current class c
    for detection in pred:
        if detection[2][0] == c:
            detections.append(detection)

    for true_box in gt:
        for jj in range(len(true_box[1])):
            if true_box[1][jj][0] == c:
                ground_truths.append([true_box[0],[c, true_box[1][jj][1:]]])

    # find the amount of bboxes for each training example
    # Counter here finds how many ground truth bboxes we get
    # for each training example, so let's say img 0 has 3,
    # img 1 has 5 then we will obtain a dictionary with:
    # amount_bboxes = {0:3, 1:5}
    amount_bboxes = Counter([gtr[0] for gtr in ground_truths])

    # We then go through each key, val in this dictionary
    # and convert to the following (w.r.t same example):
    # ammount_bboxes = {0:torch.tensor[0,0,0], 1:torch.tensor[0,0,0,0,0]}
    for key, val in amount_bboxes.items():
        amount_bboxes[key] = torch.zeros(val)

    # sort by box probabilities which is index 2
    detections.sort(key=lambda x: x[1], reverse=True)
    TP = torch.zeros((len(detections)))
    FP = torch.zeros((len(detections)))
    total_true_bboxes = len(ground_truths)

    # If none exists for this class then we can safely skip
    if total_true_bboxes == 0:
        continue

    for detection_idx, detection in enumerate(detections):
        # Only take out the ground_truths that have the same
        # training idx as detection
        ground_truth_img = [
            bbox for bbox in ground_truths if bbox[0] == detection[0]
        ]

        num_gts = len(ground_truth_img)
        best_iou = 0

        for idx, gtr in enumerate(ground_truth_img):
            iou = intersection_over_union(
                torch.tensor(detection[2][1]),
                torch.tensor(gtr[1][1]),
                box_format="midpoint",
            )

In [ ]:
detections

In [ ]:
for detection_idx, detection in enumerate(detections):
    print(detection_idx,detection[0])
#     ground_truth_img = [
#             bbox for bbox in ground_truths if bbox[0] == detection[0]
#         ]

In [ ]:
detections.sort(key=lambda x: x[1], reverse=True)
TP = torch.zeros((len(detections)))
FP = torch.zeros((len(detections)))
total_true_bboxes = len(ground_truths)
for detection_idx, detection in enumerate(detections):
    # Only take out the ground_truths that have the same
    # training idx as detection
    ground_truth_img = [
        bbox for bbox in ground_truths if bbox[0] == detection[0]
    ]

    num_gts = len(ground_truth_img)
    best_iou = 0

    for idx, gt in enumerate(ground_truth_img):
        iou = intersection_over_union(
            torch.tensor(detection[3:]),
            torch.tensor(gt[3:]),
            box_format="midpoint",
        )

        if iou > best_iou:
            best_iou = iou
            best_gt_idx = idx

    if best_iou > 0.5:
        # only detect ground truth detection once
        if amount_bboxes[detection[0]][best_gt_idx] == 0:
            # true positive and add this bounding box to seen
            TP[detection_idx] = 1
            amount_bboxes[detection[0]][best_gt_idx] = 1
        else:
            FP[detection_idx] = 1

    # if IOU is lower then the detection is a false positive
    else:
        FP[detection_idx] = 1

In [ ]:
ground_truths =[]
# for c in range(12):
c=0
for true_box in gt:
    for jj in range(len(true_box[1])):
        if true_box[1][jj][0] == 2:
            ground_truths.append([true_box[0],[c, true_box[1][jj][1:]]])

In [ ]:
len(ground_truths)

In [ ]:
amount_bboxes = Counter([gtr[0] for gtr in ground_truths])

In [ ]:
for key, val in amount_bboxes.items():
    amount_bboxes[key] = torch.zeros(val)

In [ ]:
def mean_average_precision(
    pred_boxes, true_boxes, iou_threshold=0.5, box_format="corners", num_classes=4
):


    # list storing all AP for respective classes
    average_precisions = []

    # used for numerical stability later on
    epsilon = 1e-6

    for c in range(num_classes):
        detections = []
        ground_truths = []

        # Go through all predictions and targets,
        # and only add the ones that belong to the
        # current class c
        for detection in pred_boxes:
            if detection[2][0] == c:
                detections.append(detection)

        for true_box in true_boxes:
            for jj in range(len(true_box[1])):
                if true_box[1][jj][0] == c:
                    ground_truths.append([true_box[0],[c, true_box[1][jj][1:]]])

        # find the amount of bboxes for each training example
        # Counter here finds how many ground truth bboxes we get
        # for each training example, so let's say img 0 has 3,
        # img 1 has 5 then we will obtain a dictionary with:
        # amount_bboxes = {0:3, 1:5}
        amount_bboxes = Counter([gtr[0] for gtr in ground_truths])

        # We then go through each key, val in this dictionary
        # and convert to the following (w.r.t same example):
        # ammount_bboxes = {0:torch.tensor[0,0,0], 1:torch.tensor[0,0,0,0,0]}
        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        # sort by box probabilities which is index 2
        detections.sort(key=lambda x: x[1], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)
        
        # If none exists for this class then we can safely skip
        if total_true_bboxes == 0:
            continue
        
        for detection_idx, detection in enumerate(detections):
            # Only take out the ground_truths that have the same
            # training idx as detection
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]

            num_gts = len(ground_truth_img)
            best_iou = 0

            for idx, gtr in enumerate(ground_truth_img):
                iou = intersection_over_union(
                    torch.tensor(detection[2][1]),
                    torch.tensor(gtr[1][1]),
                    box_format=box_format,
                )

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                # only detect ground truth detection once
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    # true positive and add this bounding box to seen
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1

            # if IOU is lower then the detection is a false positive
            else:
                FP[detection_idx] = 1

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        average_precisions.append(torch.trapz(precisions, recalls))

    return sum(average_precisions) / len(average_precisions)

In [ ]:
iou =mean_average_precision(pred, gt, iou_threshold=0.5, box_format="midpoint", num_classes=4)

In [ ]:
iou

In [13]:
%ls

detection.ipynb  mask/  resized/  selected/  yolov5s.pt  yolox_m/
faster/          new/   retina/   yolov5/    yolox/      yolox_s/


## YOLOV5

In [9]:
folder_path = 'bike'
w = 1000
h = 300
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf=0.33
data_list = os.listdir(folder_path)
# annot_list = os.listdir(annot_path)

for i in range(len(data_list)):
    img_path = os.path.join(folder_path, data_list[i])
#     anot_path = os.path.join(annot_path, annot_list[i])
    image = imread(img_path) 
    dim = (w, h)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    results = model(image)
    results.save()

Using cache found in /home/sahar/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-2-15 torch 1.8.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients, 16.5 GFLOPs
Adding AutoShape... 
Saved 1 image to runs/detect/exp
Saved 1 image to runs/detect/exp2
Saved 1 image to runs/detect/exp3
Saved 1 image to runs/detect/exp4
Saved 1 image to runs/detect/exp5
Saved 1 image to runs/detect/exp6
Saved 1 image to runs/detect/exp7
Saved 1 image to runs/detect/exp8
Saved 1 image to runs/detect/exp9
Saved 1 image to runs/detect/exp10
Saved 1 image to runs/detect/exp11
Saved 1 image to runs/detect/exp12
Saved 1 image to runs/detect/exp13
Saved 1 image to runs/detect/exp14


In [1]:
%ls

detection.ipynb  mask_seg/   resized/  selected/   yolox/
faster/          new/        retina/   yolov5/     yolox_m/
mask/            pointrend/  runs/     yolov5s.pt  yolox_s/


In [7]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.4
image = imread('resized/pre41.png') 
# dim = (w, h)
# image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
results = model(image)
results.save()

Using cache found in /home/sahar/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-2-15 torch 1.8.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients, 16.5 GFLOPs
Adding AutoShape... 
Saved 1 image to runs/detect/exp60
